In [4]:
##### Batch processing of all 10 files downloaded as JSON #####
import json
import pandas as pd
import os
import matplotlib.pyplot as plt

input_json_path = r'D:\Challenge\drug-label-0001-of-0010.json'
ouput_csv_path = r'D:\Challenge\Zip_data\AZ_R'

if not os.path.exists(ouput_csv_path):
    os.mkdir(ouput_csv_path)

ff = os.listdir(input_json_path)

for filess in ff:

    with open(os.path.join(input_json_path, filess)) as f:
        data = json.load(f)


    print(type(data))

    data_sz = json.dumps(data)

    no_of_record = len(data['results'])
    print(no_of_record)
    df_drug = pd.DataFrame(columns=['effective_time', 'spl_product_data_elements', 'active_ingredient',
                                    'route','brand_name', 'substance_name'])

    for rec in range(0, no_of_record):
        if 'effective_time' in data['results'][rec]:
            eft = data['results'][rec]["effective_time"]

        if 'spl_product_data_elements' in data['results'][rec]:
            spl = data['results'][rec]["spl_product_data_elements"]
        if 'active_ingredient' in data['results'][rec]:
            ag = data['results'][rec]["spl_product_data_elements"]
        else:
            ag = 'None'
        if 'route' in data['results'][rec]['openfda']:
            r = data['results'][rec]['openfda']["route"]
        else:
            r = 'NA'
        if 'brand_name' in data['results'][rec]['openfda']:
            bn = data['results'][rec]['openfda']["brand_name"]
        else:
            bn = 'NA'
        if 'substance_name' in data['results'][rec]['openfda']:
            sn = data['results'][rec]['openfda']["substance_name"]
        else:
            sn = 'NA'



        df_drug.columns = ['effective_time', 'spl_product_data_elements', 'active_ingredient',
                                              'route', 'brand_name', 'substance_name']
        df_drug = df_drug.append({'effective_time': eft,
                                  'spl_product_data_elements': spl,
                                  'active_ingredient': ag,
                                  'route': r,
                                  'brand_name': bn,
                                  'substance_name': sn}, ignore_index=True)

    df_drug.to_csv(os.path.join(ouput_csv_path,filess+'.csv'),
                   encoding='utf-8', index=False)
print(df_drug.head(10))

<class 'dict'>
20000
<class 'dict'>
20000
<class 'dict'>
20000
<class 'dict'>
20000
<class 'dict'>
20000
<class 'dict'>
20000
<class 'dict'>
20000
<class 'dict'>
20000
<class 'dict'>
20000
<class 'dict'>
5266
  effective_time                          spl_product_data_elements  \
0       20200117  [SOLMATE Broad Spectrum SPF 4 Avobenzone, Octo...   
1       20200101  [Malathion Malathion ISOPROPYL ALCOHOL TERPINE...   
2       20201204  [Sleep Aid Nighttime Diphenhydramine HCl DIPHE...   
3       20200507  [Penicillin V Potassium Penicillin V Potassium...   
4       20200508  [Hand Sanitizer ETHYL ALCOHOL TROLAMINE ALOE V...   
5       20181026  [STOOL SOFTENER DOCUSATE SODIUM DOCUSATE SODIU...   
6       20200511  [INSTANT HAND SANITIZER INSTANT HAND SANITIZER...   
7       20200512  [Alcohol Antiseptic 80 ALCOHOL GLYCERIN HYDROG...   
8       20200512  [365 Everyday Value Be Well Cough Ease for Kid...   
9       20190725  [Stomach Relief Ultra Bismuth subsalicylate BI...   

         

In [5]:
import json
import pandas as pd
import os

def extract_element_from_json(obj, path):
    '''
    Extracts an element from a nested dictionary or
    a list of nested dictionaries along a specified path.
    If the input is a dictionary, a list is returned.
    If the input is a list of dictionary, a list of lists is returned.
    obj - list or dict - input dictionary or list of dictionaries
    path - list - list of strings that form the path to the desired element
    '''
    def extract(obj, path, ind, arr):
        '''
            Extracts an element from a nested dictionary
            along a specified path and returns a list.
            obj - dict - input dictionary
            path - list - list of strings that form the JSON path
            ind - int - starting index
            arr - list - output list
        '''
        key = path[ind]
        if ind + 1 < len(path):
            if isinstance(obj, dict):
                if key in obj.keys():
                    extract(obj.get(key), path, ind + 1, arr)
                else:
                    arr.append(None)
            elif isinstance(obj, list):
                if not obj:
                    arr.append(None)
                else:
                    for item in obj:
                        extract(item, path, ind, arr)
            else:
                arr.append(None)
        if ind + 1 == len(path):
            if isinstance(obj, list):
                if not obj:
                    arr.append(None)
                else:
                    for item in obj:
                        arr.append(item.get(key, None))
            elif isinstance(obj, dict):
                arr.append(obj.get(key, None))
            else:
                arr.append(None)
        return arr
    if isinstance(obj, dict):
        return extract(obj, path, 0, [])
    elif isinstance(obj, list):
        outer_arr = []
        for item in obj:
            outer_arr.append(extract(item, path, 0, []))
        return outer_arr


if __name__ == '__main__':

    output_csv_path = r'D:\AZ-Challenge\Zip_data'
    json_path = r'D:\AZ-Challenge\drug-label-0001-of-0010.json'

    if not os.path.exists(output_csv_path):
        os.mkdir(output_csv_path)

    with open(os.path.join(json_path,'drug-label-0001-of-0010.json')) as f:
      data = json.load(f)

    df_drug = pd.read_csv(r'D:\smote\AZ_R\drug-label-0001-of-0010.json.csv')
    print(df_drug.dtypes)

    df_drug ['effective_time'] = df_drug['effective_time'].apply(str)
    print(df_drug.dtypes)
    no_of_record = len(df_drug['effective_time'])


    df_drug['Time'] = pd.to_datetime(df_drug.effective_time)


    df_drug['Year'] = df_drug.Time.dt.year

    print(df_drug.dtypes)


    for idx, row in df_drug.iterrows():
        if extract_element_from_json(data, ["results", "openfda", "substance_name"])[idx] is None:
            df_drug.loc[idx, 'num_substance'] = '0'
        else:
            df_drug.loc[idx, 'num_substance'] = len(extract_element_from_json(data, ["results", "openfda", "substance_name"])[idx])


    df_drug.to_csv(os.path.join(output_csv_path,'drug-label-0001-of-0010.json_R1.csv'),index=False)

    n_by_year = df_drug.groupby("Year").count()
    print(n_by_year)

    df_year=pd.read_csv(os.path.join(output_csv_path,'drug-label-0001-of-0010.json_R1.csv'))

    print(df_year.groupby('Year').size())
    df_ingredients = df_year.copy()

    df_ingredients['sum_of_ingredients_by_year'] = df_ingredients.groupby("Year")['num_substance'].transform(lambda x: x.sum())

    df_ingredients.to_csv(os.path.join(output_csv_path, 'drug-label-0001-of-0010.json_ingredient_R1_year.csv'))






effective_time                int64
spl_product_data_elements    object
active_ingredient            object
route                        object
brand_name                   object
substance_name               object
dtype: object
effective_time               object
spl_product_data_elements    object
active_ingredient            object
route                        object
brand_name                   object
substance_name               object
dtype: object
effective_time                       object
spl_product_data_elements            object
active_ingredient                    object
route                                object
brand_name                           object
substance_name                       object
Time                         datetime64[ns]
Year                                  int64
dtype: object
      effective_time  spl_product_data_elements  active_ingredient  route  \
Year                                                                        
1993               1

In [6]:
import json
import pandas as pd
import os

def extract_element_from_json(obj, path):
    '''
    Extracts an element from a nested dictionary or
    a list of nested dictionaries along a specified path.
    If the input is a dictionary, a list is returned.
    If the input is a list of dictionary, a list of lists is returned.
    obj - list or dict - input dictionary or list of dictionaries
    path - list - list of strings that form the path to the desired element
    '''
    def extract(obj, path, ind, arr):
        '''
            Extracts an element from a nested dictionary
            along a specified path and returns a list.
            obj - dict - input dictionary
            path - list - list of strings that form the JSON path
            ind - int - starting index
            arr - list - output list
        '''
        key = path[ind]
        if ind + 1 < len(path):
            if isinstance(obj, dict):
                if key in obj.keys():
                    extract(obj.get(key), path, ind + 1, arr)
                else:
                    arr.append(None)
            elif isinstance(obj, list):
                if not obj:
                    arr.append(None)
                else:
                    for item in obj:
                        extract(item, path, ind, arr)
            else:
                arr.append(None)
        if ind + 1 == len(path):
            if isinstance(obj, list):
                if not obj:
                    arr.append(None)
                else:
                    for item in obj:
                        arr.append(item.get(key, None))
            elif isinstance(obj, dict):
                arr.append(obj.get(key, None))
            else:
                arr.append(None)
        return arr
    if isinstance(obj, dict):
        return extract(obj, path, 0, [])
    elif isinstance(obj, list):
        outer_arr = []
        for item in obj:
            outer_arr.append(extract(item, path, 0, []))
        return outer_arr


if __name__ == '__main__':

    output_csv_path = r'D:\Challenge\Zip_data'
    json_path = r'D:\Challenge\drug-label-0001-of-0010.json'

    if not os.path.exists(output_csv_path):
        os.mkdir(output_csv_path)

    with open(os.path.join(json_path,'drug-label-0010-of-0010.json')) as f:
      data = json.load(f)

    df_drug = pd.read_csv(r'D:\smote\AZ_R\drug-label-0010-of-0010.json.csv')
    print(df_drug.dtypes)

    df_drug ['effective_time'] = df_drug['effective_time'].apply(str)
    print(df_drug.dtypes)
    no_of_record = len(df_drug['effective_time'])


    df_drug['Time'] = pd.to_datetime(df_drug.effective_time)


    df_drug['Year'] = df_drug.Time.dt.year

    print(df_drug.dtypes)


    for idx, row in df_drug.iterrows():
        if extract_element_from_json(data, ["results", "openfda", "substance_name"])[idx] is None:
            df_drug.loc[idx, 'num_substance'] = '0'
        else:
            df_drug.loc[idx, 'num_substance'] = len(extract_element_from_json(data, ["results", "openfda", "substance_name"])[idx])


    df_drug.to_csv(os.path.join(output_csv_path,'drug-label-0010-of-0010.json_R1_route.csv'),index=False)

    n_by_route = df_drug.groupby("route").count()
    print(n_by_route)
    
    

#     df_year=pd.read_csv(os.path.join(output_csv_path,'drug-label-0010-of-0010.json_R1_route.csv'))

#     print(df_year.groupby('Year').size())
#     df_ingredients = df_year.copy()

#     df_ingredients['sum_of_ingredients_by_year'] = df_ingredients.groupby("Year")['num_substance'].transform(lambda x: x.sum())

#     df_ingredients.to_csv(os.path.join(output_csv_path, 'drug-label-0001-of-0010.json_ingredient_R1_route.csv'))



effective_time                int64
spl_product_data_elements    object
active_ingredient            object
route                        object
brand_name                   object
substance_name               object
dtype: object
effective_time               object
spl_product_data_elements    object
active_ingredient            object
route                        object
brand_name                   object
substance_name               object
dtype: object
effective_time                       object
spl_product_data_elements            object
active_ingredient                    object
route                                object
brand_name                           object
substance_name                       object
Time                         datetime64[ns]
Year                                  int64
dtype: object
                                                    effective_time  \
route                                                                
['AURICULAR (OTIC)']              

In [7]:
n_by_route.to_csv(os.path.join(output_csv_path,'drug-label-0010-of-0010.json_R1_route_count.csv'),index=False)